In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LogisticRegression

In [ ]:
fd = pd.read_csv('features.csv')
fd_test = pd.read_csv('features_test.csv')

Признаки по событиям игры

In [ ]:
real_features = ['dif_boots_count', 'fr_boots_count', 'first_blood_team', 'dif_bottle_time', 'fr_bottle_time',\
                 'dif_flying_courier_time', 'fr_flying_courier_time', 'dif_tpscroll_count', 'fr_tpscroll_count',\
                 'dif_deaths', 'fr_deaths', 'dif_gold', 'fr_gold', 'dif_xp', 'fr_xp', 'dif_lh', 'fr_lh', 'dif_gold_2max',\
                 'fr_gold_2max', 'dif_xp_2max', 'fr_xp_2max', 'dif_lh_2max', 'fr_lh_2max', 'dif_gold_ch', 'fr_gold_ch',\
                 'dif_xp_ch', 'fr_xp_ch', 'start_time']

cat_features = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']

for f in [fd, fd_test]:
    f['first_blood_team'].fillna(0.5, inplace=True)
    f['radiant_bottle_time'].fillna(300, inplace=True)
    f['dire_bottle_time'].fillna(300, inplace=True)
    f['radiant_flying_courier_time'].fillna(300, inplace=True)
    f['dire_flying_courier_time'].fillna(300, inplace=True)

for f in [fd, fd_test]:
    eps = 10 ** -6
    f['r_deaths'] = f[['r{}_deaths'.format(i+1) for i in range(5)]].values.sum(axis=1)
    f['d_deaths'] = f[['d{}_deaths'.format(i+1) for i in range(5)]].values.sum(axis=1)
    f['r_gold'] = f[['r{}_gold'.format(i+1) for i in range(5)]].values.sum(axis=1)
    f['d_gold'] = f[['d{}_gold'.format(i+1) for i in range(5)]].values.sum(axis=1)
    f['r_xp'] = f[['r{}_xp'.format(i+1) for i in range(5)]].values.sum(axis=1)
    f['d_xp'] = f[['d{}_xp'.format(i+1) for i in range(5)]].values.sum(axis=1)
    f['r_lh'] = f[['r{}_lh'.format(i+1) for i in range(5)]].values.sum(axis=1)
    f['d_lh'] = f[['d{}_lh'.format(i+1) for i in range(5)]].values.sum(axis=1)    
    f['r_gold_2max'] = np.sort(f[['r{}_gold'.format(i+1) for i in range(5)]].values, axis=1)[:, 3:5].sum(axis=1)
    f['d_gold_2max'] = np.sort(f[['d{}_gold'.format(i+1) for i in range(5)]].values, axis=1)[:, 3:5].sum(axis=1)
    f['r_xp_2max'] = np.sort(f[['r{}_xp'.format(i+1) for i in range(5)]].values, axis=1)[:, 3:5].sum(axis=1)
    f['d_xp_2max'] = np.sort(f[['d{}_xp'.format(i+1) for i in range(5)]].values, axis=1)[:, 3:5].sum(axis=1)
    f['r_lh_2max'] = np.sort(f[['r{}_lh'.format(i+1) for i in range(5)]].values, axis=1)[:, 3:5].sum(axis=1)
    f['d_lh_2max'] = np.sort(f[['d{}_lh'.format(i+1) for i in range(5)]].values, axis=1)[:, 3:5].sum(axis=1)
    f['r_gold_pred'] = f[['r{}_gold_pred'.format(i+1) for i in range(5)]].values.sum(axis=1)
    f['d_gold_pred'] = f[['d{}_gold_pred'.format(i+1) for i in range(5)]].values.sum(axis=1)
    f['r_xp_pred'] = f[['r{}_xp_pred'.format(i+1) for i in range(5)]].values.sum(axis=1)
    f['d_xp_pred'] = f[['d{}_xp_pred'.format(i+1) for i in range(5)]].values.sum(axis=1)
    
    f['dif_deaths'] = f.r_deaths - f.d_deaths
    f['fr_deaths'] = f.dif_deaths / np.sqrt(f.r_deaths + f.d_deaths + eps)
    f['dif_gold'] = f.r_gold - f.d_gold
    f['fr_gold'] = f.dif_gold / np.sqrt(f.r_gold + f.d_gold + eps)
    f['dif_xp'] = f.r_xp - f.d_xp
    f['fr_xp'] = f.dif_xp / np.sqrt(f.r_xp + f.d_xp + eps)    
    f['dif_lh'] = f.r_lh - f.d_lh
    f['fr_lh'] = f.dif_lh / np.sqrt(f.r_lh + f.d_lh + eps)
    f['dif_gold_2max'] = f.r_gold_2max - f.d_gold_2max
    f['fr_gold_2max'] = f.dif_gold_2max / np.sqrt(f.r_gold_2max + f.d_gold_2max + eps)
    f['dif_xp_2max'] = f.r_xp_2max - f.d_xp_2max
    f['fr_xp_2max'] = f.dif_xp_2max / np.sqrt(f.r_xp_2max + f.d_xp_2max + eps)    
    f['dif_lh_2max'] = f.r_lh_2max - f.d_lh_2max
    f['fr_lh_2max'] = f.dif_lh_2max / np.sqrt(f.r_lh_2max + f.d_lh_2max + eps)
    f['dif_gold_ch'] = f.dif_gold - (f.r_gold_pred - f.d_gold_pred)
    f['fr_gold_ch'] = f.dif_gold_ch / np.sqrt(np.abs(f.dif_gold) + np.abs(f.r_gold_pred - f.d_gold_pred) + eps)
    f['dif_xp_ch'] = f.dif_xp - (f.r_xp_pred - f.d_xp_pred)
    f['fr_xp_ch'] = f.dif_xp_ch / np.sqrt(np.abs(f.dif_xp) + np.abs(f.r_xp_pred - f.d_xp_pred) + eps)
    
    f['dif_bottle_time'] = f['radiant_bottle_time'] - f['dire_bottle_time']
    f['fr_bottle_time'] = f.dif_bottle_time / np.sqrt(f.radiant_bottle_time + f.dire_bottle_time + 146 + eps)
    f['dif_flying_courier_time'] = f['radiant_flying_courier_time'] - f['dire_flying_courier_time']
    f['fr_flying_courier_time'] = f.dif_flying_courier_time / np.sqrt(f.radiant_flying_courier_time\
                                                                      + f.dire_flying_courier_time + 146 + eps)
    f['dif_tpscroll_count'] = f['radiant_tpscroll_count'] - f['dire_tpscroll_count']
    f['fr_tpscroll_count'] = f.dif_tpscroll_count / np.sqrt(f.radiant_tpscroll_count + f.dire_tpscroll_count + eps)
    f['dif_boots_count'] = f['radiant_boots_count'] - f['dire_boots_count']
    f['fr_boots_count'] = f.dif_boots_count / np.sqrt(f.radiant_boots_count + f.dire_boots_count + eps)
    
    for col in f.columns:
        if col not in (real_features + cat_features + ['radiant_win', 'match_id', 'lobby_type']):
            del f[col]

In [ ]:
def pick_to_prob(X_pick, y, n_folds=20, syn=1):
    if syn not in [1, 2]:
        raise ValueError('syn must be 1 or 2')
    if syn == 1:
        n_samples, n_features = X_pick.shape
        pr = np.zeros((n_samples, n_features), dtype=float)
        #pr = sp.csr_matrix((n_samples, n_features), dtype=float)
        neg_y = 1 - y
        kf = KFold(n_samples, shuffle=True, n_folds=n_folds)
        for train_ind, test_ind in kf:
            for i in range(0, n_features):
                ind_pos = test_ind[np.where(X_pick[test_ind, i] == 1)[0]]
                ind_neg = test_ind[np.where(X_pick[test_ind, i] == -1)[0]]
                counts = np.sum(X_pick[train_ind, i] != 0)
                successes = np.sum(y[train_ind[np.where(X_pick[train_ind, i] == 1)[0]]])\
                            +np.sum(neg_y[train_ind[np.where(X_pick[train_ind, i] == -1)[0]]])
                pr[ind_pos, i] = (successes + 1) / (counts + 2)
                pr[ind_neg, i] = - (successes + 1) / (counts + 2)
    else:
        n_samples = X_pick.shape[0]
        n_features = X_pick.shape[1] * (X_pick.shape[1] + 1) // 2
        #pr = np.zeros((n_samples, n_features), dtype=float)
        pr = sp.csr_matrix((n_samples, n_features), dtype=float)
        neg_y = 1 - y
        kf = KFold(n_samples, shuffle=True, n_folds=n_folds)
        for train_ind, test_ind in kf:
            cur = 0
            for i in range(0, X_pick.shape[1]):
                for j in range(i, X_pick.shape[1]):
                    ind_pos = test_ind[np.where(np.logical_and((X_pick[test_ind, i] == 1), (X_pick[test_ind, j] == 1)))[0]]
                    ind_neg = test_ind[np.where(np.logical_and((X_pick[test_ind, i] == -1), (X_pick[test_ind, j] == -1)))[0]]
                    counts = np.sum(X_pick[train_ind, i] * X_pick[train_ind, j] > 0.5)
                    successes = np.sum(y[train_ind[np.where(X_pick[train_ind, i] * X_pick[train_ind, j] > 0.5)[0]]])
                    pr[ind_pos, cur] = (successes + 1) / (counts + 2)
                    pr[ind_neg, cur] = - (successes + 1) / (counts + 2)
                    cur += 1
    
    return pr

def pick_to_prob_test(X_pick, X_pick_test, y, syn=1):
    if syn not in [1, 2]:
        raise ValueError('syn must be 1 or 2')
    if syn == 1:
        n_samples, n_features = X_pick_test.shape
        pr = np.zeros((n_samples, n_features), dtype=float)
        neg_y = 1 - y
        for i in range(0, n_features):
            ind_pos = np.where(X_pick_test[:, i] == 1)[0]
            ind_neg = np.where(X_pick_test[:, i] == -1)[0]
            counts = np.sum(X_pick[:, i] != 0)
            successes = np.sum(y[np.where(X_pick[:, i] == 1)[0]]) + np.sum(neg_y[np.where(X_pick[:, i] == -1)[0]])
            pr[ind_pos, i] = (successes + 1) / (counts + 2)
            pr[ind_neg, i] = - (successes + 1) / (counts + 2)
    else:
        n_samples = X_pick_test.shape[0]
        n_features = X_pick_test.shape[1] * (X_pick_test.shape[1] + 1) // 2
        #pr = np.zeros((n_samples, n_features), dtype=float)
        pr = sp.csr_matrix((n_samples, n_features), dtype=float)
        neg_y = 1 - y
        kf = KFold(n_samples, shuffle=True, n_folds=n_folds)
        for train_ind, test_ind in kf:
            cur = 0
            for i in range(0, X_pick_test.shape[1]):
                for j in range(i, X_pick_test.shape[1]):
                    ind_pos = np.where(np.logical_and((X_pick_test[:, i] == 1), (X_pick_test[:, j] == 1)))[0]
                    ind_neg = np.where(np.logical_and((X_pick_test[:, i] == -1), (X_pick_test[:, j] == -1)))[0]
                    counts = np.sum(X_pick[:, i] * X_pick[:, j] > 0.5)
                    successes = np.sum(y[np.where(X_pick[:, i] * X_pick[:, j] > 0.5)[0]])
                    pr[ind_pos, cur] = (successes + 1) / (counts + 2)
                    pr[ind_neg, cur] = - (successes + 1) / (counts + 2)
                    cur += 1
    
    return pr   

Пики героев (мешок слов)

In [ ]:
N = np.max(fd[cat_features].values)
X_pick = np.zeros((fd.shape[0], N))
X_pick_test = np.zeros((fd_test.shape[0], N))
for i, match_id in enumerate(fd.index):
    for p in range(5):
        X_pick[i, fd.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, fd.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
for i, match_id in enumerate(fd_test.index):
    for p in range(5):
        X_pick_test[i, fd_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, fd_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
X_pick = X_pick[:, X_pick.std(axis=0) != 0]
X_pick_test = X_pick_test[:, X_pick_test.std(axis=0) != 0]

Учет винрейтов героев

In [ ]:
X_pick_prob = pick_to_prob(X_pick, fd.radiant_win.values)
X_pick_test_prob = pick_to_prob_test(X_pick, X_pick_test, fd.radiant_win.values)

scl = StandardScaler(with_mean=False)
scl.fit(np.vstack((X_pick_prob, X_pick_test_prob)))
X_pick_prob = scl.transform(X_pick_prob)
X_pick_test_prob = scl.transform(X_pick_test_prob)

One-hot-encoding лобби

In [ ]:
fd.loc[fd.lobby_type == 7, 'lobby_type'] = 2
fd_test.loc[fd_test.lobby_type == 7, 'lobby_type'] = 2
enc = OneHotEncoder(sparse=False)
enc.fit(fd.lobby_type.values[:, np.newaxis])
lobby = enc.transform(fd.lobby_type.values[:, np.newaxis])
lobby_test = enc.transform(fd_test.lobby_type.values[:, np.newaxis])

In [ ]:
for f in [fd, fd_test]:
    for col in real_features:
        f[col] = (f[col] - f[col].mean()) / f[col].std()

Также схожий результат дало С=400

In [ ]:
lr = LogisticRegression()
params = {
    'penalty': ['l2'],
    'C': [0.4],
}
clf = GridSearchCV(estimator=lr, param_grid=params, scoring='log_loss', cv=5, n_jobs=2)
clf.fit(np.hstack((X_pick_prob, fd[real_features].values, lobby)), fd['radiant_win'].values)
answer = clf.predict_proba(np.hstack((X_pick_test_prob, fd_test[real_features].values, lobby_test)))

In [ ]:
clf.best_score_

In [ ]:
output = pd.DataFrame(data={'match_id': fd_test['match_id'], 'radiant_win': answer[:, 1]})
output.to_csv('output.csv', index=None)

Что можно добавить:
* Оставить индивидуальные показатели
* Отсортировать индивидуальные показатели по мощи
* Добавить признак level
* Синергия/антисинергия
* Найти лучший скейлинг
* Добавить степени признаков
* Для каждого лобби делать свое предсказание